In [ ]:
import numpy as np
import pandas as pd
from coxph_fitter import CoxPHFitter
import os

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import lifelines
import copy
from scipy.spatial import distance
from sklearn.impute import IterativeImputer
import json

In [ ]:
import random

rs = 42

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_torch()

In [ ]:
smd_threshold = 0.1

#### Utility Functions

In [ ]:
def clean_outliers(data):
    cols = list(data)
    for col in cols:
        min_value = data[col].quantile(0.01)
        max_value = data[col].quantile(0.99)
        data[col][data[col] < min_value] = None
        data[col][data[col] > max_value] = None
    return data

def quantize_bmi(cell):
    if cell < 18.5:
        return 0
    elif cell <= 24.9:
        return 1
    elif cell <= 29.9:
        return 2
    elif cell <= 34.9:
        return 3
    elif cell <= 39:
        return 4
    else:
        return 5
    
def get_status(cell):
    if cell == 'DIED':
        return 1
    else:
        return 0

def cal_weights(event, logits_treatment, stabilized=True, clip = True):
    ones_idx, zeros_idx = np.where(event == 1), np.where(event == 0)
    p_T = len(ones_idx[0]) / (len(ones_idx[0]) + len(zeros_idx[0]))
    if stabilized:
        # stabilized weights:   treated_w.sum() + controlled_w.sum() ~ N
        treated_w, controlled_w = p_T / logits_treatment[ones_idx], (1 - p_T) / (1. - logits_treatment[zeros_idx])
    else:
        # standard IPTW:  treated_w.sum() + controlled_w.sum() > N
        treated_w, controlled_w = 1. / logits_treatment[ones_idx], 1. / (1. - logits_treatment[zeros_idx])

    if clip:
        # treated_w = np.clip(treated_w, a_min=1e-06, a_max=100)
        # controlled_w = np.clip(controlled_w, a_min=1e-06, a_max=100)
        amin = np.quantile(np.concatenate((treated_w, controlled_w)), 0.01)
        amax = np.quantile(np.concatenate((treated_w, controlled_w)), 0.99)
        print('Using IPTW trim [{}, {}]'.format(amin, amax))
        treated_w = np.clip(treated_w, a_min=amin, a_max=amax)
        controlled_w = np.clip(controlled_w, a_min=amin, a_max=amax)

    treated_w, controlled_w = np.reshape(treated_w, (len(treated_w), 1)), np.reshape(controlled_w,
                                                                                     (len(controlled_w), 1))
    return treated_w, controlled_w


def get_day(cell):
    if cell == np.inf:
        return cell
    return int(cell[4:])

def get_window_lists(cell):
    day = get_day(cell)
    l = []
    for i in range(0, day+1):
        l.append('day_' + str(i))
    return l  

def get_window_lists_adj(cell, no_censor_window=None):
    day = get_day(cell)
    l = []
    for i in range(0, day+1+no_censor_window):
        if i < 29: # only 28 days available
            l.append('day_' + str(i))
    return l  

def check_balance_before_IPTW(X, all_vars):
    feature_treatment = X[X['T'] == 1][all_vars]
    feature_control = X[X['T'] == 0][all_vars]

    treatment_mean = feature_treatment.mean(0)
    treatment_std = feature_treatment.std(0)

    control_mean = feature_control.mean(0)
    control_std = feature_control.std(0)

    SMD = np.abs(treatment_mean-control_mean) / np.sqrt((treatment_std**2 + control_std**2)/2)

    print('Number of unbalanced covariates: ' + str((SMD>smd_threshold).sum()))

    return SMD


def check_balance_after_matching(X, all_vars):
    feature_treatment = X[X['T'] == 1][all_vars]
    feature_control = X[X['T'] == 0][all_vars]

    treatment_mean = feature_treatment.mean(0)
    treatment_std = feature_treatment.std(0)

    control_mean = feature_control.mean(0)
    control_std = feature_control.std(0)

    SMD = np.abs(treatment_mean-control_mean) / np.sqrt((treatment_std**2 + control_std**2)/2)

    return SMD

def check_balance_after_IPTW(X, all_vars):
    feature_treatment = X[X['T'] == 1][all_vars]
    feature_control = X[X['T'] == 0][all_vars]

    cohort_treatment = X[X['T'] == 1]['weight']
    cohort_control = X[X['T'] == 0]['weight']

    weight = np.array(cohort_treatment.values)
    weight_sum = (weight).sum()
    weight_2_sum = (weight ** 2).sum()

    treatment_mean = (feature_treatment * weight[:,np.newaxis]).sum(0) / weight_sum
    treatment_std = np.sqrt((weight_sum) / (weight_sum ** 2 - weight_2_sum) * (weight[:, np.newaxis] * ((feature_treatment - treatment_mean[np.newaxis,:]) ** 2)).sum(0))

    weight = np.array(cohort_control.values)
    weight_sum = (weight).sum()
    weight_2_sum = (weight ** 2).sum()

    control_mean = (feature_control * weight[:, np.newaxis]).sum(0) / weight_sum
    control_std = np.sqrt((weight_sum) / (weight_sum ** 2 - weight_2_sum) * (weight[:, np.newaxis] * ((feature_control - control_mean[np.newaxis, :]) ** 2)).sum(0))

    SMD = np.abs(treatment_mean-control_mean) / np.sqrt((treatment_std**2 + control_std**2)/2)

    print('Number of unbalanced covariates: ' + str((SMD>smd_threshold).sum()))
    return SMD


def run_ps(df, X, T, Y):
    # estimate the propensity score
    temp = df.copy(deep=True)
    ps = LogisticRegression().fit(temp[X], temp[T]).predict_proba(temp[X])[:, 1]
    treated_w, controlled_w = cal_weights(temp[T].values, ps, stabilized=False)
    ones_idx, zeros_idx = np.where(temp[T].values == 1), np.where(temp[T].values == 0)
    weight = np.zeros(len(temp[T].values))
    weight[ones_idx] = treated_w.squeeze()
    weight[zeros_idx] = controlled_w.squeeze()
    temp['weight'] = weight
    treatment_effect = sum(temp.query("T==1")[Y]*temp.query("T==1")["weight"]) / len(temp)
    no_treatment_effect = sum(temp.query("T==0")[Y]*temp.query("T==0")["weight"]) / len(temp)
    return [treatment_effect, no_treatment_effect]

In [ ]:
# Matching Iteratively

# Utility to define caliper for Mahalanobis distance
# median absolute deviation (MAD) of the calculated distances
def mad(arr):
    med = np.median(arr)
    return np.median(np.abs(arr - med))

def calculate_propensity_scores(X, treatment_col, covariate_list):
    model = LogisticRegression()
    model.fit(X[covariate_list], X[treatment_col])
    propensity_scores = model.predict_proba(X[covariate_list])[:, 1]
    X['ps'] = propensity_scores
    return X

def match_pairs(distances, treated_idx, control_idx, N=1, caliper=None):
    matched_pairs = []
    control_pool = copy.deepcopy(control_idx)
    distance_pool = copy.deepcopy(distances)
    matches_per_treated = dict.fromkeys(treated_idx, 0)
    unmatchable = set()
    while len(control_pool) > 0:
        for i, treated in enumerate(treated_idx):
            if treated in unmatchable or matches_per_treated[treated] >= N:
                continue
            if control_pool.empty:
                break
            min_index = np.argmin(distance_pool[i])
            min_distance = distance_pool[i, min_index]
            if caliper is not None and min_distance > caliper:
                unmatchable.add(treated)
                continue
            matched_control = control_pool[min_index]
            matched_pairs.append((treated, matched_control))
            matches_per_treated[treated] += 1
            control_pool = control_pool.drop(matched_control)
            distance_pool = np.delete(distance_pool, min_index, axis=1)
        if min(val for key, val in matches_per_treated.items() if key not in unmatchable) >= N:
            break
    return matched_pairs

def iterative_matching(X, unbalanced_covariates, treatment_col, method='propensity', allow_repetition=False, use_caliper=False):
    treated_idx = X[X[treatment_col] == 1].index
    control_idx = X[X[treatment_col] == 0].index
    running_data = X.copy(deep=True)
    matched_pairs = []
    for i, covariate in enumerate(unbalanced_covariates):
        multiplier = 1
        if i == 0:
            distances = calculate_distance(running_data, treated_idx, control_idx, covariate, treatment_col, method)
            if (covariate in continuous_vars) and use_caliper:
                flat_distances = distances.flatten()
                caliper = multiplier * mad(flat_distances)
            else:
                caliper = None
            pairs = match_pairs(distances, treated_idx, control_idx, allow_repetition=allow_repetition, caliper=caliper)
            matched_pairs = pairs
        else:
            matched_indices = [idx for pair in matched_pairs for idx in pair]
            running_data = running_data.loc[matched_indices]
            treated_idx = running_data[running_data[treatment_col] == 1].index
            control_idx = running_data[running_data[treatment_col] == 0].index
            distances = calculate_distance(running_data, treated_idx, control_idx, covariate, treatment_col, method)
            if (covariate in continuous_vars) and use_caliper:
                flat_distances = distances.flatten()
                caliper = multiplier * mad(flat_distances)
            else:
                caliper = None
            pairs = match_pairs(distances, treated_idx, control_idx, allow_repetition=allow_repetition, caliper=caliper)
            matched_pairs = pairs
    return matched_pairs

In [ ]:
def calculate_distance_propensity(X, treated_idx, control_idx, covariates, treatment_col):
    X_ps = calculate_propensity_scores(X, treatment_col, covariates)
    treated_data = X_ps.loc[treated_idx, 'ps'].values.reshape(-1, 1)
    control_data = X_ps.loc[control_idx, 'ps'].values.reshape(-1, 1)
    distances = distance.cdist(treated_data, control_data, metric='mahalanobis')
    return distances, X_ps


def matching_propensity(X, unbalanced_covariates, treatment_col, N=1, allow_repetition=False, use_caliper=False):
    treated_idx = X[X[treatment_col] == 1].index
    control_idx = X[X[treatment_col] == 0].index
    running_data = X.copy(deep=True)
    matched_pairs = []
    multiplier = 1
    distances, X_ps = calculate_distance_propensity(running_data, treated_idx, control_idx, unbalanced_covariates, treatment_col)
    flat_distances = distances.flatten()
    caliper = multiplier * mad(flat_distances)
    pairs = match_pairs(distances, treated_idx, control_idx, N=N, caliper=caliper)
    matched_pairs = pairs
    return matched_pairs, X_ps


def perform_balancing_method2(X, imp_covariates, all_vars, use_caliper=True):
    treatment_col = 'T'
    # Matching Based on Important Covariates
    if len(imp_covariates) > 0:
        matched_pairs = iterative_matching(X, imp_covariates, treatment_col, method='', use_caliper=use_caliper)
        matched_indices = [idx for pair in matched_pairs for idx in pair]
        X_matched_intermediate = X.loc[matched_indices].reset_index(drop=True)
    else:
        X_matched_intermediate = X.copy(deep=True)
    # Matching Based on Propensity Scores
    matched_pairs, X_ps = matching_propensity(X_matched_intermediate, all_vars, treatment_col, use_caliper=use_caliper)
    matched_indices = [idx for pair in matched_pairs for idx in pair]
    X_matched = X_ps.loc[matched_indices].reset_index(drop=True)
    check_balance_after_matching(X_matched, all_vars)
    return X_matched


def perform_balancing_method3(X, imp_covariates, all_vars, use_caliper=True):
    treatment_col = 'T'
    # Matching Based on Propensity Scores
    balances = check_balance_after_matching(X, all_vars)
    # loop through a couple range 0 - 0.1 in interval 0.025
    Ns = [1, 2, 3, 4]
    Ns = [4]
    for N in Ns:
        threshs = [0, 0.025, 0.05, 0.075, 0.1]
        threshs = [0]
        thresh_dict = dict.fromkeys(threshs)
        for thresh in threshs:
            thresh_dict[thresh] = {}
            balances_tuples = balances[balances > thresh]
            unbalanced_covariates = list(balances_tuples.sort_values(ascending=False).index)
            matched_pairs, X_ps = matching_propensity(X, unbalanced_covariates, treatment_col, N=N, use_caliper=use_caliper)
            matched_indices = [idx for pair in matched_pairs for idx in pair]
            X_matched = X_ps.loc[matched_indices].reset_index(drop=True)
            new_balances = check_balance_after_matching(X_matched, all_vars)
            new_balances_tuples = new_balances[new_balances > smd_threshold]
            new_unbalanced_covariates = list(new_balances_tuples.sort_values(ascending=False).index)
            thresh_dict[thresh]['data'] = X_matched
            thresh_dict[thresh]['unbalanced_covars'] = new_unbalanced_covariates
            if len(new_unbalanced_covariates) == 0:
                print(f"Using Threshold of {thresh}, Using N of {N}")
                return X_matched
                break
    min_key, items = min(thresh_dict.items(), key=lambda x: len(x[1]['unbalanced_covars']))
    print(f"Using Threshold of {min_key}, Using N of {N}")
    return items['data']

In [ ]:
simplified = 1

#### eICU Load Data

In [ ]:
df_steriods_eicu = pd.read_csv("data/eICU/steroids.csv")
df_steriods_eicu['patientunitstayid'] = df_steriods_eicu['patientunitstayid'].astype(int)
df_features_eicu = pd.read_csv("data/eICU/feature.csv")
df_patient_eicu = pd.read_csv("data/eICU/eicu-database-2.0/patient.csv")
df_sepsis_eicu = pd.read_csv("data/eICU/sepsis.csv")

df_comorb_eicu = pd.read_csv("data/eICU/patient_comorbidity_score_df.csv")
df_sofa_eicu = pd.read_csv("data/eICU/eICU_sofa.csv")
df_sofa_comps_eicu = pd.read_csv("data/eICU/eICU_SOFA_score_comps.csv")

df_features_eicu = pd.merge(df_features_eicu, df_comorb_eicu[['patientunitstayid', 'comorbidity_score']], on='patientunitstayid',
                       how='left')
df_features_eicu = pd.merge(df_features_eicu, df_sofa_eicu[['patientunitstayid', 'sofa']], on='patientunitstayid',
                       how='left')
df_features_eicu = pd.merge(df_features_eicu, df_sofa_comps_eicu, on='patientunitstayid',
                       how='left')
sofa_comps = ['Respiration_score', 'Cardiovascular_score', 'CNS_score', 'Liver_score', 'Coagulation_score', 'Renal_score']
df_features_eicu[sofa_comps] = df_features_eicu[sofa_comps].fillna(0)
df_features_eicu['sofa'] = df_features_eicu['Respiration_score'] + \
                            df_features_eicu['Coagulation_score'] + \
                            df_features_eicu['Liver_score'] + \
                            df_features_eicu['Cardiovascular_score'] + \
                            df_features_eicu['CNS_score'] + \
                            df_features_eicu['Renal_score']

df_ventilation_eicu = pd.read_csv("data/eICU/sepsis_ventilation.csv")
old_cols = [f'time_{i}' for i in range(0,29)]
df_ventilation_eicu = df_ventilation_eicu[['patientunitstayid'] + old_cols]
new_cols = ['patientunitstayid'] + [f'Vent_day_{i}' for i in range(0,29)]
df_ventilation_eicu.columns = new_cols
eligible_ventilation_patients_eicu = df_ventilation_eicu[df_ventilation_eicu['Vent_day_0'] == 1]['patientunitstayid'].values
df_ventilation_eicu['patientunitstayid'] = df_ventilation_eicu['patientunitstayid'].astype(int)

# read feature_bucket_with_filled_missing
with open('data/eICU/ventilation_duration.json', 'r') as load_f:
    ventilation_duration_data_eicu = json.load(load_f)

#### MIMIC Load Data

In [ ]:
df_steriods_mim = pd.read_csv("data/MIMIC/steroids.csv")
df_steriods_mim['patientunitstayid'] = df_steriods_mim['patientunitstayid'].astype(int)
df_patient_mim = pd.read_csv("data/MIMIC/patient_info.csv")
df_patient_mim.rename(columns={'stay_id':'patientunitstayid'}, inplace=True)
df_sepsis_mim = pd.read_csv("data/MIMIC/sepsis_sofa.csv")
df_sepsis_mim.rename(columns={'icustay_id':'patientunitstayid'}, inplace=True)

df_comorb_mim = pd.read_csv("data/MIMIC/patient_comorbidity_score_df.csv")
df_sofa_comps_mim = pd.read_csv("data/MIMIC/sofa_score_comps.csv")

df_features_mim = pd.read_csv("data/MIMIC/features.csv")
df_comorb_mim = pd.merge(df_patient_mim[['subject_id', 'patientunitstayid']], df_comorb_mim, on='subject_id',
                       how='left')
df_features_mim = pd.merge(df_features_mim, df_comorb_mim[['patientunitstayid', 'comorbidity_score']], on='patientunitstayid',
                       how='left')
df_features_mim = pd.merge(df_features_mim, df_sofa_comps_mim, on='patientunitstayid',
                       how='left')
df_features_mim.rename(columns={'SOFA_score':'sofa'}, inplace=True)
df_features_mim = pd.merge(df_features_mim, df_patient_mim[['patientunitstayid', 'gender', 'age']], on='patientunitstayid',
                       how='left')
sofa_comps = ['Respiration_score', 'Cardiovascular_score', 'CNS_score', 'Liver_score', 'Coagulation_score', 'Renal_score']
df_features_mim[sofa_comps] = df_features_mim[sofa_comps].fillna(0)
df_features_mim['sofa'] = df_features_mim['Respiration_score'] + \
                            df_features_mim['Coagulation_score'] + \
                            df_features_mim['Liver_score'] + \
                            df_features_mim['Cardiovascular_score'] + \
                            df_features_mim['CNS_score'] + \
                            df_features_mim['Renal_score']

# Imputing Height and Weight
imputer = IterativeImputer(random_state=100, max_iter=10)
df_train = df_features_mim.loc[:, ["Height", "Weight"]]
imputer.fit(df_train)
df_imputed = imputer.transform(df_train)
df_features_mim[['Height', 'Weight']] = df_imputed
df_features_mim['Height'] = df_features_mim['Height'].apply(lambda x: x if x >= 48 else 48)
# height is in inches, weight is in kg
df_features_mim['BMI'] = ((df_features_mim['Weight'] * 2.20462) / (df_features_mim['Height']*df_features_mim['Height']) * 703).astype(int)
max_days = 28

# Making GCS Feature
for i in range(1, max_days+1):
    df_features_mim[f'GCS_day_{i}'] = df_features_mim[f'gcsVerbal_day_{i}'] + df_features_mim[f'gcsMotor_day_{i}'] + df_features_mim[f'gcsEye_day_{i}']

df_ventilation_mim = pd.read_csv("data/MIMIC/ventilation_info.csv")

#### Eligibility Criteria Filtration

In [ ]:
df_zx_subtypes_eicu = pd.read_csv("data/eICU/zxu_subtypes_lr_24h.csv")
df_zx_subtypes_mim = pd.read_csv("data/MIMIC/zxu_subtypes_lr_24h.csv")

In [ ]:
# Subcomponent Analysis
sub_component_analysis = 0
cv = 1
resp = 0
cns = 0
liv = 0
coag = 0
rn = 0
greater = 1
lesser = not greater

sepsis_zx_subtype = 'NONE'
# options include NONE, 1, 3 (others too small)

# Default Parameters to Stick to
worst_case_mortality = 1
treatment_strat_question = 1
biological_question = not treatment_strat_question
grace_period_hours = 0
remove_event_before_grace = 0
create_clones_before_grace_outcome = 1
create_clones = grace_period_hours
perform_censoring = 0 or (biological_question)
remove_censored = 0
include_extra_treated = 1
remove_immortal_time_treated = 1
remove_missing_feats = 0

#### eICU Data Processing

In [ ]:
sepsis_patients_eicu = df_sepsis_eicu[df_sepsis_eicu['sepsis_onset'] <= 1440]['patientunitstayid'].values

# Get rid of patient with previous steriod usage to Day 0
# Only if greater than 160
steroid_dosage = 160
df_st = df_steriods_eicu[df_steriods_eicu['day_0'] < steroid_dosage]

# Get rid of patients without sepsis
df_st = df_st[df_st['patientunitstayid'].isin(sepsis_patients_eicu)]

# Getting Patients Who Were On Mechnical Ventilation at Baseline
df_st = df_st[df_st['patientunitstayid'].isin(eligible_ventilation_patients_eicu)]

df_censor = df_st.copy(deep=True)

# Patient is treated if they have over 160 steriods within grace window from inclusion criteria
grace_period_days = int(grace_period_hours/24.0)

grace_days_consider = []
for i in range(0, 2+grace_period_days):
    grace_days_consider.append(f"day_{i}")

if biological_question:
    df_st_temp = df_st.copy(deep=True)
    df_st_temp = df_st_temp.set_index('patientunitstayid')
    df_st_temp['treatment_day']=df_st_temp.keys()[np.argmax(df_st_temp.values>=steroid_dosage,axis=1)]
    patients_to_keep = df_st_temp['treatment_day'].isin(grace_days_consider)
    df_st_temp = df_st_temp[patients_to_keep]
    df_st = df_st[patients_to_keep.values]
    df_censor = df_censor[patients_to_keep.values]
elif treatment_strat_question:
    df_st_temp = df_st.copy(deep=True)
    df_st_temp = df_st_temp.set_index('patientunitstayid')
    df_st_temp = df_st_temp[grace_days_consider]
    df_st_temp['treatment_day']=df_st_temp.keys()[np.argmax(df_st_temp.values>=steroid_dosage,axis=1)]

# If resultant is day_0, that means it is not eligible (no steroid > 160)
df_st['T'] = (df_st_temp['treatment_day'] != 'day_0').values

df_st['T'] = df_st['T'].astype(int)
df_st['treatment_day'] = df_st_temp['treatment_day'].values
df_censor['treatment_day'] = df_st_temp['treatment_day'].values

no_censor_window = 3
df_censor['window_days'] = df_censor['treatment_day'].apply(get_window_lists_adj, no_censor_window=no_censor_window)

# Window where steriod admin is ok and if steriod admin after, patient is censored
## Excluding last for coding purposes --> when deciding censoring, start at day after
censor_days = []
for i in df_censor['patientunitstayid'].values:
    df_censor_temp = df_censor[df_censor['patientunitstayid'] == i]
    window_days = df_censor_temp['window_days'].values[0]
    treatment_day = df_censor_temp['treatment_day'].values[0]
    df_censor_temp = df_censor_temp.drop(columns=window_days+['treatment_day', 'window_days', 'patientunitstayid'])
    if df_censor_temp.size == 0:
        censor_days.append(np.inf)
        continue
    else:
        if np.sum(df_censor_temp.values>=steroid_dosage) == 0:
            censor_days.append(np.inf)
            continue
        else:
            censor_day = df_censor_temp.keys()[np.argmax(df_censor_temp.values>=steroid_dosage,axis=1)].values[0]
    treated = df_st[df_st['patientunitstayid'] == i]['T'].values[0]
    censor_days.append(censor_day)

df_censor['censor'] = censor_days
df_censor = df_censor.reset_index(drop=True)
df_censor = df_censor[['patientunitstayid', 'censor']].reset_index(drop=True)

df_st = pd.merge(df_st, df_censor, on='patientunitstayid')

# Getting Mortality Information
df_pat = df_patient_eicu[['patientunitstayid', 'hospitaldischargestatus', 'hospitaldischargeoffset', 'hospitaldischargelocation']]
df_pat = df_pat.dropna(subset=['patientunitstayid', 'hospitaldischargestatus', 'hospitaldischargeoffset'])
df_pat['hospitaldischargelocation'].fillna(value='Other', inplace=True)
df_pat['hospitaldischargestatus'] = df_pat['hospitaldischargestatus'].astype('category').cat.codes
df_pat['duration_day'] = df_pat['hospitaldischargeoffset'] / (60 * 24.0)

# # Do we treat people who died after 28 days as alive? And set duration to 28 days?
df_pat.loc[df_pat['duration_day'] >= 28, 'hospitaldischargestatus'] = 0
df_pat.loc[df_pat['duration_day'] >= 28, 'duration_day'] = 28
df_pat = df_pat[df_pat['duration_day'] >= 0]

if remove_event_before_grace:
    str_day = grace_days_consider[-1]
    day = get_day(str_day)
    df_pat = df_pat[df_pat['duration_day'] >= day]

df_st_pat = pd.merge(df_st, df_pat, on='patientunitstayid')
df_st_pat['treatment_day'] = df_st_pat['treatment_day'].apply(get_day)
df_st_pat['censor'] = df_st_pat['censor'].apply(get_day)

if create_clones_before_grace_outcome:
    temp = df_st_pat[df_st_pat['duration_day'] < get_day(grace_days_consider[-1])]
    # Make treated clones
    df_untreated_to_treated = temp[temp['T'] == 0]
    df_untreated_to_treated['T'] = 1
    df_untreated_to_treated['treatment_day'] = 1
    # df_untreated_to_treated['hospitaldischargestatus'] = 0 # censor these patients?
    # Make untreated clones
    df_treated_to_untreated = temp[temp['T'] == 1]
    df_treated_to_untreated['T'] = 0
    df_treated_to_untreated['treatment_day'] = 0
    # df_treated_to_untreated['hospitaldischargestatus'] = 0 # censor these patients?
    df_st_pat = pd.concat([df_st_pat, df_untreated_to_treated, df_treated_to_untreated]).reset_index(drop=True)

# Figure out real duration with Day 0 based on treatment (steroid > 160)
df_st_pat['temp_treatment_day'] = 0
df_st_pat.loc[df_st_pat['treatment_day'] >= 1, 'temp_treatment_day'] = df_st_pat['treatment_day'] - 1
df_st_pat['duration_real'] = df_st_pat['duration_day'] - df_st_pat['temp_treatment_day'] 
# Figure out duration if censoring for each patient
df_st_pat['duration_censor'] = df_st_pat['censor'] - df_st_pat['treatment_day']
# Some patients die on the day they are treated - treat their duration as 0
df_st_pat[(df_st_pat['duration_real'] < 0) & (df_st_pat['duration_real'] > -1)] = 0
# Small number of patients have duration much smaller than treatment - weird samples
df_st_pat = df_st_pat[df_st_pat['duration_real'] > 0]

if perform_censoring:
    # Account for censoring in duration
    df_st_pat['duration_final'] = df_st_pat[['duration_censor','duration_real']].min(axis=1)
elif remove_censored:
    # Remove patients who were censored due to retreatment
    df_st_pat['duration_censor'] = df_st_pat['duration_censor'].replace(np.inf, 1000)
    df_st_pat = df_st_pat[df_st_pat.duration_real <= df_st_pat.duration_censor]
    df_st_pat['duration_final'] = df_st_pat['duration_real']
else:
    # Do not Account for censoring in duration
    df_st_pat['duration_final'] = df_st_pat['duration_real']

df_temp = df_st_pat.copy(deep=True)
df_st_pat = df_st_pat[['patientunitstayid', 'T', 'hospitaldischargestatus', 'hospitaldischargelocation', 'duration_final', 'duration_real', 'duration_day', 'duration_censor', 'treatment_day']]
df_st_pat = df_st_pat.reset_index(drop=True)

# Get Day of Ventilation Cessation
# To note, ventilation information seems to be missing as many patients don't have data before date of death
df_ventilation_eligible = df_ventilation_eicu[df_ventilation_eicu['patientunitstayid'].isin(df_st['patientunitstayid'].values)]
df_ventilation_eligible = df_ventilation_eligible.sort_values(by=['patientunitstayid']).reset_index(drop=True)

df_ventilation_info = pd.DataFrame(data=df_ventilation_eligible['patientunitstayid'].values,
                                   columns=['patientunitstayid'])
df_ventilation_info['last_val'] = df_ventilation_eligible.ffill(axis=1).iloc[:, -1]
def get_last_nonnan_col(row):
    last_nonnan_col = row.last_valid_index()
    day = int(last_nonnan_col[last_nonnan_col.rfind('_')+1:])
    return day
df_ventilation_info['last_ventilation_measurment_day'] = df_ventilation_eligible.apply(get_last_nonnan_col, axis=1)

# Assign Patients Who Have last_ventilation_measurment_day = 0 to =1
df_ventilation_info.loc[df_ventilation_info['last_ventilation_measurment_day'] == 0, 'last_ventilation_measurment_day']=1

# Pulling ventilation duration from JSON File
pats = df_ventilation_info['patientunitstayid'].values
total_duration = []
for pat in pats:
    try:
        duration = ventilation_duration_data_eicu[str(pat)]['end'][-1] / 1440
    except:
        duration = None
    total_duration.append(duration)
total_duration = np.array(total_duration)
total_duration[total_duration > 28] = 28
df_ventilation_info['exact_ventilation_duration'] = total_duration

# Other Categories are censored (set to 0 at duration)
# Mortality events are extended to worst case

df_st_pat['hospitaldischargelocation_coded'] =  df_st_pat["hospitaldischargelocation"].map({'Other External':0,
                                        'Other Hospital':0, 'Skilled Nursing Facility':1,
                                           'Home':1, 'Rehabilitation':1, 'Death':0,
                                                    'Nursing Home':1, 'Other':0})

# Worst-Outcome Censoring for those that have ICU mortality
df_st_pat.loc[df_st_pat["hospitaldischargestatus"] == 1, "duration_final"] = 28

df_st_pat = pd.merge(df_st_pat, df_ventilation_info, on=['patientunitstayid'])
df_st_pat['ventilation_status'] = 0

# Loss to Follow Up (last_ventilation_measurment_day < duration_final) are censored at last_ventilation_measurment_day
df_st_pat['exact_ventilation_duration'] = df_st_pat[['duration_real', 'exact_ventilation_duration']].min(axis=1)
df_st_pat['duration_final_rounded'] = df_st_pat['duration_real'].astype(int)
df_st_pat['ventilation_duration'] = df_st_pat['exact_ventilation_duration'].astype(int)

# Accounting for death as a competing event, if death on day of last_ventilation_measurment_day
df_st_pat.loc[(df_st_pat['ventilation_duration'] == df_st_pat['duration_final_rounded']) &
             (df_st_pat['hospitaldischargestatus'] == 1), 'ventilation_duration'] = 28

# If discharged to a better location on day of last_ventilation_measurment_day, set status to 1
df_st_pat.loc[(df_st_pat['ventilation_duration'] == df_st_pat['duration_final_rounded']) &
             (df_st_pat['hospitaldischargelocation_coded'] == 1), 'ventilation_status'] = 1

# For those that cease ventilation, set ventilation status to 1
df_st_pat.loc[(df_st_pat['ventilation_duration'] < df_st_pat['duration_final_rounded']), 'ventilation_status'] = 1

if sub_component_analysis:
    df_st_pat = pd.merge(df_st_pat, df_sofa_comps_eicu, on='patientunitstayid')
    if cv:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Cardiovascular_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Cardiovascular_score'] < 2]
    elif resp:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Respiration_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Respiration_score'] < 2]
    elif cns:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['CNS_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['CNS_score'] < 2]
    elif liv:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Liver_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Liver_score'] < 2]
    elif coag:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Coagulation_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Coagulation_score'] < 2]
    elif rn:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Renal_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Renal_score'] < 2]
    df_st_pat = df_st_pat.drop(columns=['Cardiovascular_score', 'Respiration_score', 'CNS_score', 'Liver_score', 'Coagulation_score', 'Renal_score'])
    
try:
    zx_group = int(sepsis_zx_subtype)
    subtype_patients = df_zx_subtypes_eicu[df_zx_subtypes_eicu['subtype'] == zx_group]['patientunitstayid'].values
    df_st_pat = df_st_pat[df_st_pat['patientunitstayid'].isin(subtype_patients)]
except:
    pass

df_st_pat_eicu = df_st_pat.copy(deep=True)

df_feat = df_features_eicu.copy(deep=True)
df_feat = df_feat[df_feat['patientunitstayid'].isin(df_st_pat['patientunitstayid'].values)]
feat_day_titles = ["day_" + str(i+1) for i in range(28)]

# Drop Extremely Missing Features
lymph_features = ['Lymphocyte_count_' + day_title for day_title in feat_day_titles]
df_feat.drop(columns=lymph_features, inplace=True)
time_feature_names = ['Albumin', 'ALT', 'AST', 'Bands', 'Bicarbonate', 'Bilirubin',
                 'BUN','Chloride', 'Creatinine', 'CRP', 'FiO2', 'GCS', 'Glucose', 'Heart_rate',
                 'Hemoglobin', 'INR', 'Lactate', 'Lymphocyte_percent', 'MAP',
                 'PaO2', 'Platelet', 'RDW', 'Respiratory_rate','SO2', 'Sodium','Systolic_ABP',
                 'Temperature', 'Troponin I', 'Troponin T', 'Urine', 'WBC']
baseline_feature_names = ['age', 'gender', 'BMI', 'comorbidity_score', 'sofa', 'Respiration_score',
                         'Cardiovascular_score', 'CNS_score', 'Liver_score',
                          'Coagulation_score', 'Renal_score']

# Missingness for Baseline Confounders
imp_mean = SimpleImputer(missing_values=np.NaN, strategy='mean')
df_feat.replace([np.inf, -np.inf], np.NaN, inplace=True)
df_feat['gender'] = df_feat['gender'].fillna("Missing")
one_hot_gender = pd.get_dummies(df_feat['gender'])
df_feat = df_feat.drop('gender',axis = 1)
df_feat = df_feat.join(one_hot_gender)
df_feat = df_feat.rename(columns={"Female": "gender_Female", "Male": "gender_Male",
                                 "Unknown": "gender_Unknown"})
if 'gender_Unknown' in df_feat:
    df_feat['gender_Missing'] = df_feat['gender_Unknown']
    df_feat.drop(columns=['gender_Unknown'], inplace=True)
else:
    df_feat['gender_Missing'] = 0

num_vars = ['Albumin', 'ALT', 'AST', 'Bands', 'Bicarbonate',
                     'Bilirubin', 'BUN', 'Chloride', 'Creatinine', 'CRP',
                     'FiO2', 'GCS', 'Glucose', 'Heart_rate', 'Hemoglobin',
                     'INR', 'Lactate', 'Lymphocyte_percent', 'MAP', 'PaO2',
                     'Platelet', 'RDW', 'Respiratory_rate', 'SO2', 'Sodium',
                     'Systolic_ABP', 'Temperature', 'Troponin I', 'Troponin T',
                     'Urine', 'WBC']
num_feats = ['age', 'BMI']
for i in range(0, 28):
    for var in num_vars:
        num_feats.append(f"{var}_day_{i+1}")
df_feat[num_feats] = clean_outliers(df_feat[num_feats])
df_feat['age_missing'] = df_feat['age'].isnull().astype(int)
df_feat['BMI_missing'] = df_feat['BMI'].isnull().astype(int)
df_feat['age'] = imp_mean.fit_transform(df_feat['age'].values.reshape(-1,1))[:,0]
df_feat['BMI'] = imp_mean.fit_transform(df_feat['BMI'].values.reshape(-1,1))[:,0]
df_feat['sofa'] = imp_mean.fit_transform(df_feat['sofa'].values.reshape(-1,1))[:,0]

# Quantizing BMI
df_feat['BMI_quant'] = df_feat['BMI'].apply(quantize_bmi)
df_feat['BMI_orig'] = df_feat['BMI']
df_feat['BMI'] = df_feat['BMI_quant']

# Quantizing Age
df_feat['age_quant'], age_bins = pd.qcut(df_feat['age'], 5, labels=[0,1,2,3,4], retbins=True, precision=3, duplicates='raise')
df_feat['age_orig'] = df_feat['age']
df_feat['age'] = df_feat['age_quant']
imp_median = SimpleImputer(missing_values=np.NaN, strategy='median')
df_feat['age'] = df_feat['age'].astype(float)
df_feat['age'] = imp_median.fit_transform(df_feat['age'].values.reshape(-1,1))
df_feat['age'] = df_feat['age'].astype(int)

# Missingness for Time Varying Covariates
for tv_feature in time_feature_names:
    tv_feature_all = [tv_feature + '_' + day_title for day_title in feat_day_titles]
    df_feat[tv_feature_all] = clean_outliers(df_feat[tv_feature_all])
    df_feat[tv_feature_all] = df_feat[tv_feature_all].ffill(axis = 1)
    df_feat[tv_feature_all] = imp_mean.fit_transform(df_feat[tv_feature_all])

# Point Treatments (Get Day of Treatment for Treated) 
df_confounder_list = []
base_features = ['age', 'gender_Male', 'gender_Missing', 'BMI', 'age_missing',
                 'BMI_missing', 'comorbidity_score', 'sofa', 'Respiration_score',
                         'Cardiovascular_score', 'CNS_score', 'Liver_score',
                          'Coagulation_score', 'Renal_score']
col_titles = ['patientunitstayid'] + time_feature_names + base_features


for pat, T in zip(df_st_pat['patientunitstayid'].values, df_st_pat['T'].values):
    treatment_day = df_st_pat[(df_st_pat['patientunitstayid'] == pat) & (df_st_pat['T'] == T)]['treatment_day'].values[0]
    if treatment_day == 0:
        treatment_day = 1
    tv_point_features = [feat + '_day_' + str(treatment_day) for feat in time_feature_names]
    confounders =  tv_point_features + base_features
    df_feat_temp = df_feat[df_feat['patientunitstayid'] == pat]
    df_feat_temp = df_feat_temp[['patientunitstayid'] + confounders].reset_index(drop=True)
    df_feat_temp.columns = col_titles
    df_feat_temp['T'] = T
    df_confounder_list.append(df_feat_temp)
    
df_confounders_eicu = pd.concat(df_confounder_list)

#### MIMIC Data Processing

In [ ]:
sepsis_patients_mim = df_sepsis_mim[df_sepsis_mim['hour_24'] >= 2]['patientunitstayid'].values

# Get rid of patient with previous steriod usage to Day 0
# Only if greater than 160
steroid_dosage = 160
df_st = df_steriods_mim[df_steriods_mim['day_-1'] < steroid_dosage]

# Get rid of patients without sepsis
df_st = df_st[df_st['patientunitstayid'].isin(sepsis_patients_mim)]
df_st_follow_time = df_st[['patientunitstayid', 'follow-up-time']]
df_st = df_st.drop(columns=['follow-up-time'])

df_censor = df_st.copy(deep=True)

# Patient is treated if they have over 160 steriods within grace window from inclusion criteria
grace_period_days = int(grace_period_hours/24.0)

grace_days_consider = []
for i in range(0, 2+grace_period_days):
    grace_days_consider.append(f"day_{i}")

if biological_question:
    df_st_temp = df_st.copy(deep=True)
    df_st_temp = df_st_temp.set_index('patientunitstayid')
    df_st_temp['treatment_day']=df_st_temp.keys()[np.argmax(df_st_temp.values>=steroid_dosage,axis=1)]
    patients_to_keep = df_st_temp['treatment_day'].isin(grace_days_consider)
    df_st_temp = df_st_temp[patients_to_keep]
    df_st = df_st[patients_to_keep.values]
    df_censor = df_censor[patients_to_keep.values]
elif treatment_strat_question:
    df_st_temp = df_st.copy(deep=True)
    df_st_temp = df_st_temp.set_index('patientunitstayid')
    df_st_temp = df_st_temp[grace_days_consider]
    df_st_temp['treatment_day']=df_st_temp.keys()[np.argmax(df_st_temp.values>=steroid_dosage,axis=1)]

# If resultant is day_0, that means it is not eligible (no steroid > 160)
df_st['T'] = (df_st_temp['treatment_day'] != 'day_0').values

df_st['T'] = df_st['T'].astype(int)
df_st['treatment_day'] = df_st_temp['treatment_day'].values
df_censor['treatment_day'] = df_st_temp['treatment_day'].values

no_censor_window = 3
df_censor['window_days'] = df_censor['treatment_day'].apply(get_window_lists_adj, no_censor_window=no_censor_window)

# Window where steriod admin is ok and if steriod admin after, patient is censored
## Excluding last for coding purposes --> when deciding censoring, start at day after
censor_days = []
for i in df_censor['patientunitstayid'].values:
    df_censor_temp = df_censor[df_censor['patientunitstayid'] == i]
    window_days = df_censor_temp['window_days'].values[0]
    treatment_day = df_censor_temp['treatment_day'].values[0]
    df_censor_temp = df_censor_temp.drop(columns=window_days+['treatment_day', 'window_days', 'patientunitstayid'])
    if df_censor_temp.size == 0:
        censor_days.append(np.inf)
        continue
    else:
        if np.sum(df_censor_temp.values>=steroid_dosage) == 0:
            censor_days.append(np.inf)
            continue
        else:
            censor_day = df_censor_temp.keys()[np.argmax(df_censor_temp.values>=steroid_dosage,axis=1)].values[0]
    treated = df_st[df_st['patientunitstayid'] == i]['T'].values[0]
    censor_days.append(censor_day)

df_censor['censor'] = censor_days
df_censor = df_censor.reset_index(drop=True)
df_censor = df_censor[['patientunitstayid', 'censor']].reset_index(drop=True)

df_st = pd.merge(df_st, df_censor, on='patientunitstayid')

# Getting Mortality Information
df_pat = df_patient_mim[['patientunitstayid', 'hospitaldischargeoffset', 'hospitaldischargelocation']]
df_pat = df_pat.dropna(subset=['patientunitstayid', 'hospitaldischargelocation', 'hospitaldischargeoffset'])
df_pat['hospitaldischargestatus'] = df_pat['hospitaldischargelocation'].apply(get_status)
df_pat['duration_day'] = df_pat['hospitaldischargeoffset']

# # Do we treat people who died after 28 days as alive? And set duration to 28 days?
df_pat.loc[df_pat['duration_day'] >= 28, 'hospitaldischargestatus'] = 0
df_pat.loc[df_pat['duration_day'] >= 28, 'duration_day'] = 28
df_pat = df_pat[df_pat['duration_day'] >= 0]

if remove_event_before_grace:
    str_day = grace_days_consider[-1]
    day = get_day(str_day)
    df_pat = df_pat[df_pat['duration_day'] >= day]

df_st_pat = pd.merge(df_st, df_pat, on='patientunitstayid')
df_st_pat['treatment_day'] = df_st_pat['treatment_day'].apply(get_day)
df_st_pat['censor'] = df_st_pat['censor'].apply(get_day)

if create_clones_before_grace_outcome:
    temp = df_st_pat[df_st_pat['duration_day'] < get_day(grace_days_consider[-1])]
    # Make treated clones
    df_untreated_to_treated = temp[temp['T'] == 0]
    df_untreated_to_treated['T'] = 1
    df_untreated_to_treated['treatment_day'] = 1
    # df_untreated_to_treated['hospitaldischargestatus'] = 0 # censor these patients?
    # Make untreated clones
    df_treated_to_untreated = temp[temp['T'] == 1]
    df_treated_to_untreated['T'] = 0
    df_treated_to_untreated['treatment_day'] = 0
    # df_treated_to_untreated['hospitaldischargestatus'] = 0 # censor these patients?
    df_st_pat = pd.concat([df_st_pat, df_untreated_to_treated, df_treated_to_untreated]).reset_index(drop=True)

if remove_immortal_time_treated:
    df_st_pat = pd.merge(df_st_pat, df_st_follow_time, on='patientunitstayid')
    df_st_pat.loc[df_st_pat['T'] == 0, 'follow-up-time'] = 0
    df_st_pat['follow-up-time'] = df_st_pat['follow-up-time'] / 1440
    df_st_pat['duration_day'] = df_st_pat['duration_day'] - df_st_pat['follow-up-time']
    df_st_pat = df_st_pat[df_st_pat['duration_day'] > 0]

# Figure out real duration with Day 0 based on treatment (steroid > 160)
df_st_pat['temp_treatment_day'] = 0
df_st_pat.loc[df_st_pat['treatment_day'] >= 1, 'temp_treatment_day'] = df_st_pat['treatment_day'] - 1
df_st_pat['duration_real'] = df_st_pat['duration_day'] - df_st_pat['temp_treatment_day'] 
# Figure out duration if censoring for each patient
df_st_pat['duration_censor'] = df_st_pat['censor'] - df_st_pat['treatment_day']
# Some patients die on the day they are treated - treat their duration as 0
df_st_pat[(df_st_pat['duration_real'] < 0) & (df_st_pat['duration_real'] > -1)] = 0
# Small number of patients have duration much smaller than treatment - weird samples
df_st_pat = df_st_pat[df_st_pat['duration_real'] > 0]

if perform_censoring:
    # Account for censoring in duration
    df_st_pat['duration_final'] = df_st_pat[['duration_censor','duration_real']].min(axis=1)
elif remove_censored:
    # Remove patients who were censored due to retreatment
    df_st_pat['duration_censor'] = df_st_pat['duration_censor'].replace(np.inf, 1000)
    df_st_pat = df_st_pat[df_st_pat.duration_real <= df_st_pat.duration_censor]
    df_st_pat['duration_final'] = df_st_pat['duration_real']
else:
    # Do not Account for censoring in duration
    df_st_pat['duration_final'] = df_st_pat['duration_real']

df_temp = df_st_pat.copy(deep=True)
df_st_pat = df_st_pat[['patientunitstayid', 'T', 'hospitaldischargestatus', 'hospitaldischargelocation', 'duration_final', 'duration_real', 'duration_day', 'duration_censor', 'treatment_day']]
df_st_pat = df_st_pat.reset_index(drop=True)

# Getting outcomes for menchanical ventilation
df_ventilation_mim.rename(columns={'stay_id':'patientunitstayid'}, inplace=True)
duplicate_mask = df_ventilation_mim.duplicated(subset='patientunitstayid', keep=False)
duplicate_rows = df_ventilation_mim[duplicate_mask]
unique_rows = df_ventilation_mim[~duplicate_mask]

# Combining Certain Patients data
unique_pats_in_dup = pd.unique(duplicate_rows['patientunitstayid'])
new_patient_info = []
for pat in unique_pats_in_dup:
    pat_rows = duplicate_rows[duplicate_rows['patientunitstayid'] == pat] 
    day_start = pat_rows['day_start'].values[0]
    total_time_used = np.sum(pat_rows['time_used_days'].values)
    subject_id = pat_rows['subject_id'].values[0]
    ham_id = pat_rows['hadm_id'].values[0]
    new_patient_info.append([subject_id, ham_id, pat, day_start, total_time_used])
new_df = pd.DataFrame(data=new_patient_info, columns=list(duplicate_rows))
df_ventilation_edit = pd.concat([unique_rows, new_df])
df_st_pat = pd.merge(df_st_pat, df_ventilation_edit[['patientunitstayid', 'day_start', 'time_used_days']], on=['patientunitstayid'])

# Filtering patients to only patients who have ventilation at baseline
df_st_pat = df_st_pat[df_st_pat['day_start'] == 1]
df_st_pat['ventilation_status'] = 0
df_st_pat['ventilation_duration'] = df_st_pat['time_used_days']
# Accounting for death as a competing event, if death on day of last_ventilation_measurment_day
df_st_pat.loc[(df_st_pat['hospitaldischargelocation'] == 'DIED'), 'ventilation_duration'] = 28
# If discharged to a better location, set status to 1
df_st_pat.loc[(df_st_pat['hospitaldischargelocation'] != 'DIED'), 'ventilation_status'] = 1
df_st_pat['ventilation_duration'] = df_st_pat['ventilation_duration'].clip(upper=28)
    
if sub_component_analysis:
    df_st_pat = pd.merge(df_st_pat, df_sofa_comps_mim, on='patientunitstayid')
    if cv:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Cardiovascular_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Cardiovascular_score'] < 2]
    elif resp:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Respiration_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Respiration_score'] < 2]
    elif cns:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['CNS_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['CNS_score'] < 2]
    elif liv:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Liver_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Liver_score'] < 2]
    elif coag:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Coagulation_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Coagulation_score'] < 2]
    elif rn:
        if greater:
            df_st_pat = df_st_pat[df_st_pat['Renal_score'] >= 2]
        else:
            df_st_pat = df_st_pat[df_st_pat['Renal_score'] < 2]
    df_st_pat = df_st_pat.drop(columns=['Cardiovascular_score', 'Respiration_score', 'CNS_score', 'Liver_score', 'Coagulation_score', 'Renal_score'])
    
try:
    zx_group = int(sepsis_zx_subtype)
    subtype_patients = df_zx_subtypes_mim[df_zx_subtypes_mim['subtype'] == zx_group]['patientunitstayid'].values
    df_st_pat = df_st_pat[df_st_pat['patientunitstayid'].isin(subtype_patients)]
except:
    pass


df_st_pat_mim = df_st_pat.copy(deep=True)

df_feat = df_features_mim.copy(deep=True)
feat_day_titles = ["day_" + str(i+1) for i in range(28)]

# Drop Extremely Missing Features
## Bicarbonate, CRP, GCS], [Lactate]
time_feature_names = ['Albumin', 'ALT', 'AST', 'Bands', 'Bilirubin',
                 'BUN','Chloride', 'Creatinine', 'FiO2', 'Glucose', 'Heart_rate',
                 'Hemoglobin', 'INR', 'MAP', 'Lymphocyte_count', 'Lactate', 'GCS',
                 'PaO2', 'Platelet', 'Respiratory_rate','SO2', 'Sodium','Systolic_ABP',
                 'Temperature', 'Troponin T', 'Urine', 'WBC']
baseline_feature_names = ['age', 'gender', 'BMI', 'sofa', 'Respiration_score', 'comorbidity_score'
                         'Cardiovascular_score', 'CNS_score', 'Liver_score',
                          'Coagulation_score', 'Renal_score']

# Missingness for Baseline Confounders
imp_mean = SimpleImputer(missing_values=np.NaN, strategy='mean')
df_feat.replace([np.inf, -np.inf], np.NaN, inplace=True)
df_feat['gender'] = df_feat['gender'].fillna("Missing")
one_hot_gender = pd.get_dummies(df_feat['gender'])
df_feat = df_feat.drop('gender',axis = 1)
df_feat = df_feat.join(one_hot_gender)
df_feat = df_feat.rename(columns={"Female": "gender_Female", "Male": "gender_Male",
                                 "Unknown": "gender_Unknown"})
if 'gender_Unknown' in df_feat:
    df_feat['gender_Missing'] = df_feat['gender_Unknown']
    df_feat.drop(columns=['gender_Unknown'], inplace=True)
else:
    df_feat['gender_Missing'] = 0

num_vars = ['Albumin', 'ALT', 'AST', 'Bands',
                     'Bilirubin', 'BUN', 'Chloride', 'Creatinine',
                     'FiO2', 'Glucose', 'Heart_rate', 'Hemoglobin',
                     'INR', 'MAP', 'PaO2', 'Lymphocyte_count', 'Lactate', 'GCS',
                     'Platelet', 'Respiratory_rate', 'SO2', 'Sodium',
                     'Systolic_ABP', 'Temperature', 'Troponin T',
                     'Urine', 'WBC']
num_feats = ['age', 'BMI']
for i in range(0, 28):
    for var in num_vars:
        num_feats.append(f"{var}_day_{i+1}")
df_feat[num_feats] = clean_outliers(df_feat[num_feats])
df_feat['age_missing'] = df_feat['age'].isnull().astype(int)
df_feat['BMI_missing'] = df_feat['BMI'].isnull().astype(int)
df_feat['age'] = imp_mean.fit_transform(df_feat['age'].values.reshape(-1,1))[:,0]
df_feat['BMI'] = imp_mean.fit_transform(df_feat['BMI'].values.reshape(-1,1))[:,0]
df_feat['sofa'] = imp_mean.fit_transform(df_feat['sofa'].values.reshape(-1,1))[:,0]

# Quantizing BMI
df_feat['BMI_quant'] = df_feat['BMI'].apply(quantize_bmi)
df_feat['BMI_orig'] = df_feat['BMI']
df_feat['BMI'] = df_feat['BMI_quant']

# Quantizing Age
df_feat['age_quant'] = pd.cut(df_feat['age'], bins=age_bins, labels=[0,1,2,3,4], include_lowest=True)
df_feat['age_orig'] = df_feat['age']
df_feat['age'] = df_feat['age_quant']
imp_median = SimpleImputer(missing_values=np.NaN, strategy='median')
df_feat['age'] = df_feat['age'].astype(float)
df_feat['age'] = imp_median.fit_transform(df_feat['age'].values.reshape(-1,1))
df_feat['age'] = df_feat['age'].astype(int)

# Missingness for Time Varying Covariates
for tv_feature in time_feature_names:
    tv_feature_all = [tv_feature + '_' + day_title for day_title in feat_day_titles]
    df_feat[tv_feature_all] = clean_outliers(df_feat[tv_feature_all])
    df_feat[tv_feature_all] = df_feat[tv_feature_all].ffill(axis = 1)
    df_feat[tv_feature_all] = imp_mean.fit_transform(df_feat[tv_feature_all])
    
df_feat = df_feat[df_feat['patientunitstayid'].isin(df_st_pat['patientunitstayid'].values)]
    
# Point Treatments (Get Day of Treatment for Treated) 
df_confounder_list = []
base_features = ['age', 'gender_Male', 'gender_Missing', 'BMI', 'age_missing',
                 'BMI_missing', 'sofa', 'Respiration_score', 'comorbidity_score',
                         'Cardiovascular_score', 'CNS_score', 'Liver_score',
                          'Coagulation_score', 'Renal_score']
col_titles = ['patientunitstayid'] + time_feature_names + base_features


for pat, T in zip(df_st_pat['patientunitstayid'].values, df_st_pat['T'].values):
    treatment_day = df_st_pat[(df_st_pat['patientunitstayid'] == pat) & (df_st_pat['T'] == T)]['treatment_day'].values[0]
    if treatment_day == 0:
        treatment_day = 1
    tv_point_features = [feat + '_day_' + str(treatment_day) for feat in time_feature_names]
    confounders =  tv_point_features + base_features
    df_feat_temp = df_feat[df_feat['patientunitstayid'] == pat]
    df_feat_temp = df_feat_temp[['patientunitstayid'] + confounders].reset_index(drop=True)
    df_feat_temp.columns = col_titles
    df_feat_temp['T'] = T
    df_confounder_list.append(df_feat_temp)
    
df_confounders_mim = pd.concat(df_confounder_list)

# Data Preparation

### Balancing Covariates

In [ ]:
df_eicu = pd.merge(df_confounders_eicu, df_st_pat_eicu, on=['patientunitstayid', 'T'])
df_mim = pd.merge(df_confounders_mim, df_st_pat_mim, on=['patientunitstayid', 'T'])
overlapping_columns = (list(set(list(df_mim)) & set(list(df_eicu))))
overlapping_columns.insert(0, overlapping_columns.pop(overlapping_columns.index('patientunitstayid')))
df_mim = df_mim[overlapping_columns]
df_mim['dataset'] = 0
df_eicu = df_eicu[overlapping_columns]
df_eicu['dataset'] = 1
df = pd.concat([df_mim, df_eicu])

overlapping_columns.append('dataset')
df.reset_index(drop=True, inplace=True)

keep_certain_level_sofa = 0
if keep_certain_level_sofa:
    lower_level = 8
    higher_level = 25
    df = df[(df['sofa'] >= lower_level) & (df['sofa'] <= higher_level)].reset_index(drop=True)

patient_id = ['patientunitstayid']
misc_vars = ['T', 'hospitaldischargestatus', 'duration_final', 'duration_real',
             'hospitaldischargelocation','ventilation_duration', 'ventilation_status']
cat_vars = ['gender_Male', 'dataset']
num_vars = ['Albumin', 'ALT', 'AST', 'Bands', 'Bicarbonate',
                     'Bilirubin', 'BUN', 'Chloride', 'Creatinine', 'CRP',
                     'FiO2', 'GCS', 'Glucose', 'Heart_rate', 'Hemoglobin',
                     'INR', 'Lactate', 'Lymphocyte_percent', 'MAP', 'PaO2',
                     'Platelet', 'RDW', 'Respiratory_rate', 'SO2', 'Sodium',
                     'Systolic_ABP', 'Temperature', 'Troponin I', 'Troponin T',
                     'Urine', 'WBC', 'comorbidity_score', 'sofa', 'Respiration_score',
                         'Cardiovascular_score', 'CNS_score', 'Liver_score',
                          'Coagulation_score', 'Renal_score', 'age', 'BMI']
discrete_vars = ['gender_Male', 'dataset']
continuous_vars = ['Albumin', 'ALT', 'AST', 'Bands', 'Bicarbonate',
                     'Bilirubin', 'BUN', 'Chloride', 'Creatinine', 'CRP',
                     'FiO2', 'GCS', 'Glucose', 'Heart_rate', 'Hemoglobin',
                     'INR', 'Lactate', 'Lymphocyte_percent', 'MAP', 'PaO2',
                     'Platelet', 'RDW', 'Respiratory_rate', 'SO2', 'Sodium',
                     'Systolic_ABP', 'Temperature', 'Troponin I', 'Troponin T',
                     'Urine', 'WBC', 'comorbidity_score', 'age', 'Respiration_score',
                         'Cardiovascular_score', 'CNS_score', 'Liver_score',
                          'Coagulation_score', 'Renal_score', 'BMI']

cat_vars = (list(set(list(cat_vars)) & set(list(overlapping_columns))))
num_vars = (list(set(list(num_vars)) & set(list(overlapping_columns))))
discrete_vars = (list(set(list(discrete_vars)) & set(list(overlapping_columns))))
continuous_vars = (list(set(list(continuous_vars)) & set(list(overlapping_columns))))

all_vars = num_vars + cat_vars

# Remove colzumns with no differences in values across patients
for col in all_vars:
    if col == 'dataset':
        continue
    if (df[col] == df[col][0]).all():
        df.drop(columns=[col], inplace=True)
        all_vars.remove(col)

scaler = MinMaxScaler()
df[num_vars] = scaler.fit_transform(df[num_vars])
X = df.drop(columns=['patientunitstayid', 'treatment_day'])
# X = df.copy(deep=True)

# Removing sofa from analysis bc we have components
use_vars = copy.deepcopy(all_vars)
use_vars.remove('sofa')

if remove_missing_feats:
    use_vars.remove('ALT')
    use_vars.remove('AST')
    use_vars.remove('Albumin')
    use_vars.remove('Bands')
    use_vars.remove('Troponin T')

imp_covariates = []
X_matched = perform_balancing_method3(X, imp_covariates, use_vars, use_caliper=True)
X = X_matched.copy(deep=True).reset_index(drop=True)

print(f"Number of Treated Patients: {X[X['T'] == 1].shape[0]}")
print(f"Percent of Patients Who Ceased Ventilation: {X[X['ventilation_status'] == 1].shape[0] / X.shape[0]}")

balances = check_balance_after_matching(X, use_vars)
print('Number of unbalanced covariates: ' + str((balances>smd_threshold).sum()))
balances[balances > smd_threshold]

In [ ]:
# Check the distribution of propensity scores and weights
treatment = X[X['T'] == 1]
control = X[X['T'] == 0]

plt.figure(figsize=(15, 6)) # Increase the width to better accommodate three subplots

plt.subplot(1, 3, 1) # Change subplot layout to 1 row, 3 columns
plt.hist(treatment['ps'], alpha=0.5, label='Treatment', density=True)
plt.hist(control['ps'], alpha=0.5, label='Control', density=True)
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.legend()

plt.subplot(1, 3, 3) # Change subplot layout to 1 row, 3 columns
other_var = 'Lactate'
plt.hist(treatment[other_var], alpha=0.5, label='Treatment', density=True)
plt.hist(control[other_var], alpha=0.5, label='Control', density=True)
plt.xlabel(other_var)
plt.ylabel('Density')
plt.legend()

plt.show()

#### Cumulative Incidence

In [ ]:
X['outcome'] =  X['ventilation_status']
X_treated = X[X['T'] == 1]
X_untreated = X[X['T'] == 0]

print(f"Naive 28 Day Cumulative Incidence for Treated: {X_treated[X_treated['outcome'] == 1].shape[0]/len(X_treated)}")
print(f"Naive 28 Day Cumulative Incidence for Untreated: {X_untreated[X_untreated['outcome'] == 1].shape[0]/len(X_untreated)}")
print(f"Percent of Patients Who Ceased Ventilation: {X[X['ventilation_status'] == 1].shape[0] / X.shape[0]}")

print(f"Alive Length for Treated: {np.round(X_treated['ventilation_duration'].mean(), 2)}±{np.round(X_treated['ventilation_duration'].std(), 2)}")
print(f"Alive Length for Untreated: {np.round(X_untreated['ventilation_duration'].mean(), 2)}±{np.round(X_untreated['ventilation_duration'].std(), 2)}")

#### Cox Proportional Hazard Model

In [ ]:
cph = CoxPHFitter()
features = use_vars + ['T', 'ventilation_duration', 'ventilation_status']

data_cox = X[features]
for col in list(data_cox):
    if (data_cox[col] == data_cox[col][0]).all():
        data_cox.drop(columns=[col], inplace=True)

cph.fit(data_cox, duration_col='ventilation_duration', event_col='ventilation_status',
        robust=True, fit_options={'step_size': 0.1})
cph.print_summary()

In [ ]:
# Predict median time to discharge for each treatment group
med_time = cph.predict_median(data_cox)
# Get difference in median time between the two treatment groups
treated_indices = list(data_cox[data_cox['T'] == 1].index)
untreated_indices = list(data_cox[data_cox['T'] == 0].index)
diff = np.median(med_time.iloc[untreated_indices]) - np.median(med_time.iloc[treated_indices])
print(f"Difference in Median Time Between Groups: {diff}")

In [ ]:
from lifelines import KaplanMeierFitter

In [ ]:
kmf = KaplanMeierFitter()

T = data_cox['ventilation_duration']
E = data_cox['ventilation_status']

plt.figure(figsize=(10,7))
ax = plt.gca()  # Define the axis object here

# fit the model for each treatment T and plot
for name, grouped_df in data_cox.groupby('T'):
    if name == 0:
        label = 'Untreated'
    else:
        label = 'Treated'
    kmf.fit(grouped_df['ventilation_duration'], grouped_df['ventilation_status'], label=label)
    
    # Transform the survival function and confidence intervals
    transformed_sf = 1 - kmf.survival_function_
    transformed_confidence = 1 - kmf.confidence_interval_
    
    # Plot the transformed survival function and confidence intervals
    ax.plot(transformed_sf, label=label)
    ax.fill_between(transformed_confidence.index, 
                    transformed_confidence.iloc[:, 0], 
                    transformed_confidence.iloc[:, 1], 
                    color=ax.lines[-1].get_color(), 
                    alpha=0.3)

plt.xlabel('Days', fontsize=14)  # set x-axis label
plt.ylabel('Probability of Ceasing Ventilation', fontsize=14)  # set y-axis label
plt.tick_params(axis='both', which='major', labelsize=14)

plt.ylim(-0.05, 1.05)  # set y-axis limits
plt.yticks(np.arange(0, 1.1, 0.1))  # set y-axis ticks

plt.legend().remove()
plt.show()